# Youtube Playlist 갱신을 위한 곡 insert 코드
> 다음 코드는 음악 추천 알고리즘에 따라 추출된 곡 제목을 Youtube에서 검색하여, 해당 영상을 playlist에 추가하기 위한 것이다. 과정은 다음과 같다. 

1. 음악 추천 알고리즘에서 output으로 나온 추천곡 id를 데이터셋에서 검색하여 제목을 찾아낸다.
2. 찾아낸 제목으로 youtube 검색, 추가할 곡 video의 id를 찾는다.
3. video id를 youtube api를 활용해 playlist에 추가한다. 
4. (1-3까지의 코드가 오류 없이 실행되면)웹페이지에서 playlist가 전체 반복될 때마다 새로운 곡이 추가되도록 한다. 

 - 참고 : https://developers.google.com/youtube/v3/docs/playlists/insert , https://stackoverflow.com/questions/21228815/adding-youtube-video-to-playlist-using-python            

# 1. Selenium으로 Youtube 검색기 만들기
> Selenium을 활용해 youtube 홈페이지에서 원하는 곡을 검색하고, 해당 곡 영상의 video ID를 얻어낸다.
<br>
<br>
<br>
## 0. 라이브러리 불러오기
- 중요 라이브러리
    - selenium
    - BeautifulSoup

In [70]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

## 1. Youtube 크롤링
> 서버에서 막혀 실패

In [66]:
from seleniumwire import webdriver
import time
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome('C:/Users/user/Desktop/(driver) chromedriver.exe')
driver.header_overrides = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
}
driver.get('https://www.google.com/')

time.sleep(6)
driver.close()

## 2. Google 크롤링
> 제대로 크롤링이 될까 걱정했으나 생각보다 오랜 시간이 걸리지 않았고, 비교적 간단했다. 연속적인 크롤링으로 서버를 차단당하는 일이 없도록 주의한다. time.sleep()을 빼먹지 않도록!

In [121]:
# 구글 크롤링(완료) 

from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

artist_name = "오반"
song_name = "행복"

driver = webdriver.Chrome('C:/Users/user/Desktop/(driver) chromedriver.exe')
driver.get('https://www.google.com')

time.sleep(5)

# assert 'Youtube' in driver.title
#//*[@id="query"]

search = driver.find_element_by_xpath("//*[@id='tsf']/div[2]/div[1]/div[1]/div/div[2]/input")

search.send_keys(artist_name+' '+song_name)
search.send_keys(Keys.ENTER)

time.sleep(10)

# beautifulsoup 
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

time.sleep(5)

for href in soup.find('div', {'class' : 'FGpTBd'}).find_all('a'):
    url = href.get('href')
    print(url)
    
    li = list(url)
    print(li.index('=')) #idx=31
    song_id = li[32:]
    print(song_id)
    song_id = ''.join(song_id) # list를 문자열로 변경
    print(song_id)
    
driver.close()

https://www.youtube.com/watch?v=qGh0jk-f6to
31
['q', 'G', 'h', '0', 'j', 'k', '-', 'f', '6', 't', 'o']
qGh0jk-f6to


## 3. 네이버 크롤링
> 처음에는 구글 대신 사용하려고 하였으나 이하의 단점으로 선택하지 않았다.
    1. 검색 시 동영상 탭, 검색 조건 탭, 영상 소스 탭 등 선택해야 하는 요소가 많다.
    2. Youtube에 있는 모든 동영상 중 일부만 검색된다. 개중에는 노래와 상관없는 영상도 많다.

In [87]:
# 네이버 크롤링 

q = "버즈+가시"


driver = webdriver.Chrome('C:/Users/user/Desktop/(driver) chromedriver.exe')
driver.get('https://www.naver.com/')

time.sleep(5)

# assert 'Youtube' in driver.title
#//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input

search = driver.find_element_by_xpath("//*[@id='query']")

search.send_keys(q)
search.send_keys(Keys.ENTER)

time.sleep(5)
# //*[@id="lnb"]/div/div[1]/ul/li[2]/a

video = driver.find_element_by_xpath("//*[@id='lnb']/div/div[1]/ul/li[6]/a")
video.click()

search_options = driver.find_element_by_xpath("//*[@id='_search_option_btn']")
search_options.click()

select = driver.find_element_by_xpath('//*[@id="snb"]/div/ul/li[1]/a')
select.click()

watch_views = driver.find_element_by_xpath('//*[@id="snb"]/div/ul/li[1]/div/ul/li[3]/a')
watch_views.click()

# beautifulsoup 
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

time.sleep(5)

for link in soup.find_all('a', {'class' : 'title'}):
    song_id = link.attrs['href']
    print(song_id)
    

driver.close()

https://www.youtube.com/watch?v=gn4m-HLgQOo
https://www.youtube.com/watch?v=5JsHAWuCdTM
https://tv.naver.com/v/987136
https://www.youtube.com/watch?v=c0zJrHvW8K0
https://www.youtube.com/watch?v=iu9cjTE6m7I
https://www.youtube.com/watch?v=zZOojGpt-DA
https://www.youtube.com/watch?v=R-H3tcywlOA
https://www.youtube.com/watch?v=RnWaNg_Kxek
https://www.youtube.com/watch?v=mNZvtYajjuE
https://www.youtube.com/watch?v=sYBW6Cx2DEM
https://www.youtube.com/watch?v=Lboznwv_Fhg
https://www.youtube.com/watch?v=0A-VMJasTSw
https://tv.naver.com/v/20873
https://www.youtube.com/watch?v=V5S2Vbsq5vs
https://tv.naver.com/v/1029444
https://tv.naver.com/v/764190


In [43]:
webdriver.__version__

'3.14.1'

# 2. Youtube API로 playlist에 곡 추가하기

In [25]:
# 라이브러리 불러오기

import httplib2
import os
import sys

from googleapiclient import discovery
from googleapiclient import errors
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client import tools